<a href="https://colab.research.google.com/github/lucash-h/SENG474/blob/main/DecisionTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

In [87]:
file_path = '/content/drive/MyDrive/Colab Notebooks/SENG 474/Assignment1/spambase_augmented.csv'
df = pd.read_csv(file_path)

In [88]:
print(df.head())

   0.000  0.640  0.640.1  0.000.1  0.320  0.000.2  0.000.3  0.000.4  0.000.5  \
0   0.21   0.28     0.50      0.0   0.14     0.28     0.21     0.07     0.00   
1   0.06   0.00     0.71      0.0   1.23     0.19     0.19     0.12     0.64   
2   0.00   0.00     0.00      0.0   0.63     0.00     0.31     0.63     0.31   
3   0.00   0.00     0.00      0.0   0.63     0.00     0.31     0.63     0.31   
4   0.00   0.00     0.00      0.0   1.85     0.00     0.00     1.85     0.00   

   0.000.6  ...  0.000.816  0.000.817  0.000.818  0.000.819  0.000.820  \
0     0.94  ...       0.00        0.0        0.0       0.00       0.00   
1     0.25  ...       0.06        0.0        0.0       0.12       0.06   
2     0.63  ...       0.00        0.0        0.0       0.00       0.00   
3     0.63  ...       0.00        0.0        0.0       0.00       0.00   
4     0.00  ...       0.00        0.0        0.0       0.00       0.00   

   0.000.821  0.000.822  0.000.823  0.000.824  1  
0       0.00       0.00

In [89]:
#now to split data into vectors
#each has 48 features of the frequency of some word
#this dataset has the original 57 features and a new feature for every pair of the 48 words and for any pair of words, the new feature is the sum of those words frequencies
#that means that there is 2^48 + 58 features? subract like pairs
#column asp is # 1186 so yea
#so vectors will be 1 * 1186 what the fuck



In [90]:
import numpy as np

np.shape(df)

(4600, 1186)

In [91]:

#split into training/test data  ***might be worth adding labels ie last col***
#maybe randomize later
#3640-> train
#960 -> test
x = df.iloc[:,:-1] #features
y = df.iloc[:,-1] #label/last col

x_train = x[:3640]
y_train = y[:3640]
print(np.shape(x_train))
print(np.shape(y_train))

x_test = x[3640:]
y_test = y[3640:]
print(np.shape(x_test))
print(np.shape(y_test))

(3640, 1185)
(3640,)
(960, 1185)
(960,)


In [92]:
from sklearn.model_selection import train_test_split

In [93]:
x = df.iloc[:,:-1] #features
y = df.iloc[:,-1] #label/last col

In [94]:
print(x.shape)
print(y.shape)

(4600, 1185)
(4600,)


In [95]:
print(x.iloc[0])

0.000        0.21
0.640        0.28
0.640.1      0.50
0.000.1      0.00
0.320        0.14
             ... 
0.000.820    0.00
0.000.821    0.00
0.000.822    0.00
0.000.823    0.00
0.000.824    0.00
Name: 0, Length: 1185, dtype: float64


In [96]:
print(y.iloc[0])

1


In [97]:
x_train, x_test, y_train, y_test = train_test_split(x,y, train_size=0.8, random_state=None, shuffle=True, stratify=None)

In [98]:
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(3680, 1185)
(3680,)
(920, 1185)
(920,)


In [99]:
from sklearn.tree import DecisionTreeClassifier

In [100]:
import matplotlib.pyplot as plt

In [101]:
from sklearn.tree import plot_tree
from sklearn import tree


In [102]:
#left child -> left_child_index = dtc.tree_.children_left[0]
#right child -> right_child_index = dtc.tree_.children_right[0]

In [103]:
import copy

Create mega tree <br>
Go through every node from the bottom <br>
at each node, remove it and it's children and compare the accuracy of the full tree with and without it <br>
record the accuracy(maybe regardless, would make life easier)<br>
Go until you reach the root node<br>
then find the tree with the greatest accuracy <br>

algo (node) :
Assuming you already have the accuracy measure for tree<br>
curr = node <br>
curr.left = null<br>
curr.right = null <br>
curr = null <br>

### should probably do it recursively but general idea is there I think -> for reduced error pruning

In [104]:
from sklearn import tree
from sklearn.base import clone

In [105]:
dtc = DecisionTreeClassifier(criterion="gini", random_state=0).fit(x_train, y_train)  # Exclude the target column name
cloned = clone(dtc).fit(x_train,y_train)



In [106]:
accuracy_before_pruning = dtc.score(x_test, y_test)
print("Accuracy before pruning:", accuracy_before_pruning)

Accuracy before pruning: 0.9195652173913044


In [107]:
def is_leaf(model, index):
  if ((model.tree_.children_left[index] == -1) & (model.tree_.children_right[index] == -1)) :
    return True
  else:
    return False

In [108]:
print(is_leaf(dtc, 1),
is_leaf(dtc, 320),
is_leaf(dtc, 160)
)

False True True


In [109]:
def prune_tree(model, index, x_value, y_value):
     # Base case: If the node is a leaf, do nothing
    if is_leaf(dtc, index):
      return

    # Children check
    prune_tree(model, model.tree_.children_left[index], x_value, y_value)
    prune_tree(model, model.tree_.children_right[index], x_value, y_value)

    # Set both children to -1 to "remove" the subtree
    model.tree_.children_left[index] = -1
    model.tree_.children_right[index] = -1

    #CHANGED MIGHT HAVE BLOWN IT UP
    #keep track of children
    prev_left_child = model.tree_.children_left[index]
    prev_right_child = model.tree_.children_right[index]

    #measure accuracy here for og tree
    current_accuracy = model.score(x_value, y_value)
    #make them leaf nodes
    model.tree_.children_left[index] = -1
    model.tree_.children_right[index] = -1

    new_accuracy = model.score(x_value, y_value)

    #measure accuracy for changed tree

    if(new_accuracy < current_accuracy): #if pruning doesn't improve accuracy add children back
      model.tree_.children_left[index] = prev_left_child
      model.tree_.children_right[index] = prev_right_child
    return model

In [110]:
def print_model(model):
  plt.figure(figsize=(23, 23))
  plot_tree(model, filled=True)
  plt.show()

In [111]:
new = prune_tree(cloned, 0, x_train, y_train)
acc1 = dtc.score(x_test, y_test)
acc2 = new.score(x_test, y_test)

In [112]:
print(acc1, acc2)

0.9195652173913044 0.6
